In [9]:
import requests
import json
from pprint import pprint
from nytkey import key
import time
import datetime as dt

In [ ]:
#Return all bestseller lists 

key = key

url = f"https://api.nytimes.com/svc/books/v3/lists/names.json?api-key={key}"

print(url)

lists = requests.get(url).json()
pprint(lists)

In [ ]:
#Store a list of active bestseller lists
bslists = []

for x in range(55):
    if "2019" in lists['results'][x]['newest_published_date']:
        bslists.append(lists['results'][x]['display_name'])
        
print(bslists)

In [ ]:
#Retrieve and store a list of the current "Young Adult Hardcover bestsellers"
url2 = f"https://api.nytimes.com/svc/books/v3/lists/current/Young-Adult-Hardcover.json?api-key={key}"

yacurrent = requests.get(url2).json()
pprint(yacurrent)

In [ ]:
#Retrieve and store all books on any bestseller list currently

currentlists = []

for bs in bslists:
    currentlisturl = f"https://api.nytimes.com/svc/books/v3/lists/current/{bs}.json?api-key={key}"
    bs = bs.replace(" ", "-")
    currentlists.append(requests.get(currentlisturl).json())
    time.sleep(1)
    
pprint(currentlists)    

In [ ]:
#Figure out which dates to query for the weekly dates (decided to do 3 years)
yadates = []

latestYA = yacurrent["results"]['bestsellers_date']
latestYA = dt.datetime.strptime(latestYA, "%Y-%m-%d")
latestYA = dt.datetime.date(latestYA)
print(latestYA)

for x in range(156):
    pastdate = latestYA - (dt.timedelta(days=7))*x
    yadates.append(pastdate)

print(yadates)

In [ ]:
#A functional list of all the dates the bestseller list was published on for the last year
datesforreal = []

for ya in yadates:
    temp = ya.strftime("%Y-%m-%d")
    datesforreal.append(temp)

print(datesforreal)

In [ ]:
#Retrieve the data
yeardata = []

for d in datesforreal:
    yaurl = f"https://api.nytimes.com/svc/books/v3/lists/{d}/Young-Adult-Hardcover.json?api-key={key}"
    data = requests.get(yaurl).json()
    yeardata.append(data)
    time.sleep(6)
    
print(yeardata)

In [ ]:
titles = []
authors = []
publishers = []
#rank = []
isbn10 = []
isbn13 = []
descriptions = []
book_images = []
for x in range(156):
    for y in range(10):
        try:
            titles.append(yeardata[x]['results']['books'][y]['title'])
            authors.append(yeardata[x]['results']['books'][y]['author'])
            publishers.append(yeardata[x]['results']['books'][y]['publisher'])
            #rank.append(yeardata[x]['results']['books'][y]['rank'])
            isbn10.append(yeardata[x]['results']['books'][y]['primary_isbn10'])
            isbn13.append(yeardata[x]['results']['books'][y]['primary_isbn13'])
            descriptions.append((yeardata[x]['results']['books'][y]['description']))
            book_images.append(yeardata[x]['results']['books'][y]['book_image'])
        except KeyError:
            print("Data Not Found")
                      
print(titles)

In [4]:
import pandas as pd
import numpy as np

In [ ]:
#Put the list into a dataframe
nyt_df = pd.DataFrame({'Title' : titles, 'Author': authors, 'Publisher' : publishers, 'ISBN-10' : isbn10, 'ISBN-13' : isbn13, "Description" : descriptions, 'Image' : book_images})
nyt_df.head()



In [ ]:
#Save the dataframe as a csv for future reference
nyt_df.to_csv("nytyabestsellers.csv", index=False, header=True)

In [3]:
#Read the file back in
file = "nytyabestsellers.csv"
nyt_df = pd.read_csv(file)
nyt_df

,Title,Author,Publisher,ISBN-10,ISBN-13,Description,Image
0,FIVE FEET APART,Rachael Lippincott with Mikki Daughtry and Tob...,Simon & Schuster,1534437339,9781534437333,"Stella and Will are in love, but they can't ge...",https://s1.nyt.com/du/books/images/97815344373...
1,THE HATE U GIVE,Angie Thomas,Balzer + Bray,0062498533,9780062498533,A 16-year-old girl sees a police officer kill ...,https://s1.nyt.com/du/books/images/97800624985...
2,ON THE COME UP,Angie Thomas,Balzer + Bray,None,9780062498564,Bri strives to become a hip-hop star despite n...,https://s1.nyt.com/du/books/images/97800624985...
3,DEAR EVAN HANSEN: THE NOVEL,"Val Emmich with Steven Levenson, Benj Pasek & ...",Poppy,None,9780316420235,When one of Evan's letters to himself is accid...,https://s1.nyt.com/du/books/images/97803164202...
4,"THE FORK, THE WITCH, AND THE WORM",Christopher Paolini with Angela Paolini,Knopf,1984894862,9781984894861,Three stories set in the magical world of Alag...,https://s1.nyt.com/du/books/images/97819848948...
5,CHILDREN OF BLOOD AND BONE,Tomi Adeyemi,Holt,1250170974,9781250170972,Zélie fights to restore magic to the land of O...,https://s1.nyt.com/du/books/images/97812501709...
6,SHOUT,Laurie Halse Anderson,Viking,0670012106,9780670012107,The author’s memoir in free verse.,https://s1.nyt.com/du/books/images/97806700121...
7,TWO CAN KEEP A SECRET,Karen M McManus,Delacorte,1524714720,9781524714727,Ellery Corcoran investigates the mysteries of ...,https://s1.nyt.com/du/books/images/97815247147...
8,ONE OF US IS LYING,Karen M McManus,Delacorte,1524714682,9781524714680,"For five students, a detour into detention end...",https://s1.nyt.com/du/books/images/97815247146...
9,THE POET X,Elizabeth Acevedo,HarperTeen,0062662805,9780062662804,A coming-of-age story told in verse.,https://s1.nyt.com/du/books/images/97800626628...


In [ ]:
title_count = nyt_df['Title'].value_counts()
print(title_count)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(title_count, bins = 15)
plt.title("Time on Bestseller List 4/2016-4/2019")
plt.ylabel("Number of Weeks")
plt.xlabel("Number of Titles")

In [ ]:
fig, ax = plt.subplots()
ax.boxplot(title_count)
#ax.boxplot(author_count)
plt.title("Time on Bestseller List 4/2016-4/2019")
plt.ylabel("Number of Weeks")
plt.xlabel("Title")

In [ ]:
author_count = nyt_df['Author'].value_counts()
print(author_count)

In [ ]:
plt.hist(author_count, bins = 15)

In [ ]:
plt.boxplot(author_count)
plt.title("Time on Bestseller List 4/2016-4/2019")
plt.ylabel("Number of Weeks")
plt.xlabel("Author")

In [ ]:
publisher_count = nyt_df['Publisher'].value_counts()
print(publisher_count)

In [ ]:
plt.hist(publisher_count)

In [ ]:
plt.boxplot(publisher_count)

In [ ]:
#Replace "none" in ISBN column w/ Nan:
nyt_df['ISBN-10'].replace('None', np.nan, inplace=True)
nyt_df['ISBN-13'].replace('None', np.nan, inplace=True)
nyt_df['ISBN-13'].count()

In [5]:
from grkeys import grkey
from grkeys import userid

In [9]:
#Use isbns to link to goodreads
chunk1 = nyt_df['ISBN-13'].tolist()

#############################################
#  IGNORE EVERYTHING ELSE, IT DOESN'T WORK
#############################################
#chunk2 = isbn13[51:100]
#chunk3 = isbn13[101:156]

grurl = f"https://www.goodreads.com/book/review_counts.json?isbns[]={chunk1}&key={grkey}"
#grurl2 = f"https://www.goodreads.com/book/isbns[]={chunk2}/review_counts.json?key={grkey}"
#grurl3 = f"https://www.goodreads.com/book/isbns[]={chunk3}/review_counts.json?key={grkey}"

reviews1 = requests.get(grurl).json()
#reviews2 = requests.get(grurl2).json()
#reviews3 = requests.get(grurl3).json()

pprint(reviews1)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [18]:
#Get goodreads review info given isbn

reviews = []

for isbn in chunk1:
    grurl = f"https://www.goodreads.com/book/review_counts.json?isbns[]={isbn}&key={grkey}"
    reviews1 = requests.get(grurl).json()
    reviews.append(reviews1)
    time.sleep(1)

In [19]:
pprint(reviews)

[{'books': [{'average_rating': '4.20',
             'id': 38926328,
             'isbn': '1534437339',
             'isbn13': '9781534437333',
             'ratings_count': 1536,
             'reviews_count': 6059,
             'text_reviews_count': 336,
             'work_ratings_count': 19197,
             'work_reviews_count': 63372,
             'work_text_reviews_count': 3131}]},
 {'books': [{'average_rating': '4.54',
             'id': 32075671,
             'isbn': '0062498533',
             'isbn13': '9780062498533',
             'ratings_count': 232933,
             'reviews_count': 78593,
             'text_reviews_count': 32889,
             'work_ratings_count': 294280,
             'work_reviews_count': 84449,
             'work_text_reviews_count': 42354}]},
 {'books': [{'average_rating': '4.37',
             'id': 35887567,
             'isbn': '0062498568',
             'isbn13': '9780062498564',
             'ratings_count': 8538,
             'reviews_count': 40234,
 

             'work_reviews_count': 284119,
             'work_text_reviews_count': 15294}]},
 {'books': [{'average_rating': '4.26',
             'id': 40036542,
             'isbn': '1984848453',
             'isbn13': '9781984848451',
             'ratings_count': 38,
             'reviews_count': 726,
             'text_reviews_count': 13,
             'work_ratings_count': 38,
             'work_reviews_count': 735,
             'work_text_reviews_count': 13}]},
 {'books': [{'average_rating': '4.45',
             'id': 35396841,
             'isbn': '0062662805',
             'isbn13': '9780062662804',
             'ratings_count': 1978,
             'reviews_count': 11234,
             'text_reviews_count': 646,
             'work_ratings_count': 20145,
             'work_reviews_count': 60807,
             'work_text_reviews_count': 4796}]},
 {'books': [{'average_rating': '4.06',
             'id': 39088507,
             'isbn': '0316420239',
             'isbn13': '9780316420235'

             'text_reviews_count': 1585,
             'work_ratings_count': 10576,
             'work_reviews_count': 34416,
             'work_text_reviews_count': 1802}]},
 {'books': [{'average_rating': '4.21',
             'id': 34728667,
             'isbn': '1250170974',
             'isbn13': '9781250170972',
             'ratings_count': 63950,
             'reviews_count': 253001,
             'text_reviews_count': 12685,
             'work_ratings_count': 77056,
             'work_reviews_count': 284119,
             'work_text_reviews_count': 15294}]},
 {'books': [{'average_rating': '4.06',
             'id': 32682118,
             'isbn': '1524714682',
             'isbn13': '9781524714680',
             'ratings_count': 10629,
             'reviews_count': 57763,
             'text_reviews_count': 1779,
             'work_ratings_count': 104329,
             'work_reviews_count': 284303,
             'work_text_reviews_count': 16148}]},
 {'books': [{'average_rating': '4.45'

             'isbn13': '9781534404953',
             'ratings_count': 2824,
             'reviews_count': 25884,
             'text_reviews_count': 930,
             'work_ratings_count': 3226,
             'work_reviews_count': 27121,
             'work_text_reviews_count': 1067}]},
 {'books': [{'average_rating': '4.21',
             'id': 34728667,
             'isbn': '1250170974',
             'isbn13': '9781250170972',
             'ratings_count': 63950,
             'reviews_count': 253001,
             'text_reviews_count': 12685,
             'work_ratings_count': 77056,
             'work_reviews_count': 284119,
             'work_text_reviews_count': 15294}]},
 {'books': [{'average_rating': '4.06',
             'id': 39088507,
             'isbn': '0316420239',
             'isbn13': '9780316420235',
             'ratings_count': 9198,
             'reviews_count': 30930,
             'text_reviews_count': 1585,
             'work_ratings_count': 10576,
             'work_re

 {'books': [{'average_rating': '4.45',
             'id': 35396841,
             'isbn': '0062662805',
             'isbn13': '9780062662804',
             'ratings_count': 1978,
             'reviews_count': 11234,
             'text_reviews_count': 646,
             'work_ratings_count': 20145,
             'work_reviews_count': 60807,
             'work_text_reviews_count': 4796}]},
 {'books': [{'average_rating': '4.06',
             'id': 38355098,
             'isbn': '1481481967',
             'isbn13': '9781481481960',
             'ratings_count': 7118,
             'reviews_count': 31560,
             'text_reviews_count': 1722,
             'work_ratings_count': 8056,
             'work_reviews_count': 34003,
             'work_text_reviews_count': 1953}]},
 {'books': [{'average_rating': '4.54',
             'id': 32075671,
             'isbn': '0062498533',
             'isbn13': '9780062498533',
             'ratings_count': 232933,
             'reviews_count': 78595,
    

...

             'isbn13': '9780062310668',
             'ratings_count': 123648,
             'reviews_count': 283785,
             'text_reviews_count': 13384,
             'work_ratings_count': 142517,
             'work_reviews_count': 318852,
             'work_text_reviews_count': 15592}]},
 {'books': [{'average_rating': '4.09',
             'id': 15745753,
             'isbn': '1250012570',
             'isbn13': '9781250012579',
             'ratings_count': 651731,
             'reviews_count': 1288354,
             'text_reviews_count': 48278,
             'work_ratings_count': 727103,
             'work_reviews_count': 1416771,
             'work_text_reviews_count': 60337}]},
 {'books': [{'average_rating': '4.68',
             'id': 26073150,
             'isbn': '1619634465',
             'isbn13': '9781619634466',
             'ratings_count': 14942,
             'reviews_count': 49986,
             'text_reviews_count': 3167,
             'work_ratings_count': 222076,
      

             'id': 25494343,
             'isbn': '1442468351',
             'isbn13': '9781442468351',
             'ratings_count': 117627,
             'reviews_count': 333987,
             'text_reviews_count': 13026,
             'work_ratings_count': 131602,
             'work_reviews_count': 361454,
             'work_text_reviews_count': 15316}]},
 {'books': [{'average_rating': '3.88',
             'id': 27190613,
             'isbn': '0553522310',
             'isbn13': '9780553522310',
             'ratings_count': 24982,
             'reviews_count': 111407,
             'text_reviews_count': 4836,
             'work_ratings_count': 31562,
             'work_reviews_count': 140123,
             'work_text_reviews_count': 6501}]},
 {'books': [{'average_rating': '3.99',
             'id': 25890355,
             'isbn': '0451472403',
             'isbn13': '9780451472403',
             'ratings_count': 5917,
             'reviews_count': 18267,
             'text_reviews_count'

             'work_text_reviews_count': 31240}]},
 {'books': [{'average_rating': '4.08',
             'id': 18692431,
             'isbn': '0553496646',
             'isbn13': '9780553496642',
             'ratings_count': 243904,
             'reviews_count': 512813,
             'text_reviews_count': 28694,
             'work_ratings_count': 276842,
             'work_reviews_count': 567476,
             'work_text_reviews_count': 33844}]},
 {'books': [{'average_rating': '4.07',
             'id': 22328546,
             'isbn': '0062310631',
             'isbn13': '9780062310637',
             'ratings_count': 312578,
             'reviews_count': 685818,
             'text_reviews_count': 20970,
             'work_ratings_count': 434707,
             'work_reviews_count': 947977,
             'work_text_reviews_count': 38373}]},
 {'books': [{'average_rating': '4.18',
             'id': 18460392,
             'isbn': '0385755880',
             'isbn13': '9780385755887',
             

             'work_ratings_count': 142517,
             'work_reviews_count': 318852,
             'work_text_reviews_count': 15592}]},
 {'books': [{'average_rating': '4.07',
             'id': 22328546,
             'isbn': '0062310631',
             'isbn13': '9780062310637',
             'ratings_count': 312578,
             'reviews_count': 685818,
             'text_reviews_count': 20970,
             'work_ratings_count': 434707,
             'work_reviews_count': 947977,
             'work_text_reviews_count': 38373}]},
 {'books': [{'average_rating': '4.04',
             'id': 17838528,
             'isbn': '1481404547',
             'isbn13': '9781481404549',
             'ratings_count': 26977,
             'reviews_count': 90737,
             'text_reviews_count': 4540,
             'work_ratings_count': 29859,
             'work_reviews_count': 96580,
             'work_text_reviews_count': 5267}]},
 {'books': [{'average_rating': '3.47',
             'id': 27272506,
        

             'reviews_count': 128533,
             'text_reviews_count': 6533,
             'work_ratings_count': 45686,
             'work_reviews_count': 140178,
             'work_text_reviews_count': 7380}]},
 {'books': [{'average_rating': '4.36',
             'id': 25614492,
             'isbn': '0399160302',
             'isbn13': '9780399160301',
             'ratings_count': 72809,
             'reviews_count': 244352,
             'text_reviews_count': 12053,
             'work_ratings_count': 87021,
             'work_reviews_count': 280935,
             'work_text_reviews_count': 14410}]},
 {'books': [{'average_rating': '3.77',
             'id': 25817407,
             'isbn': '0062391992',
             'isbn13': '9780062391995',
             'ratings_count': 6831,
             'reviews_count': 21008,
             'text_reviews_count': 1401,
             'work_ratings_count': 46398,
             'work_reviews_count': 112909,
             'work_text_reviews_count': 5884}]},
 

In [41]:
(reviews[1]['books'][0]['id'])

32075671

In [44]:
gr_id = []
isbn13 = []
rating = []
ratings_count = []
reviews_count = []

for x in range(1560):
    gr_id.append(reviews[x]['books'][0]['id'])
    isbn13.append(reviews[x]['books'][0]['isbn13'])
    rating.append(reviews[x]['books'][0]['average_rating'])
    ratings_count.append(reviews[x]['books'][0]['ratings_count'])
    reviews_count.append(reviews[x]['books'][0]['reviews_count'])
    
reviews_df = pd.DataFrame({"ID" : gr_id, "ISBN-13" : isbn13, "Average Rating" : rating, "Ratings Count" : ratings_count, "Reviews Count" : reviews_count})    

reviews_df.to_csv("GoodReadsReviews.csv")

In [6]:
file = "GoodReadsReviews.csv"
reviews_df = pd.read_csv(file)

uisbn = reviews_df['ISBN-13'].unique()
uisbn = uisbn.tolist()

In [11]:
#Pull more information from goodreads, including publication date
bookinfo = []

for isbn in uisbn:
    grurl2 = f"https://www.goodreads.com/book/isbn/{isbn}?format=json&user_id={userid}"
    #print(grurl2)
    info = requests.get(grurl2).json()
    bookinfo.append(info)
    time.sleep(1)
pprint(bookinfo)

[{'reviews_widget': '<style>\n'
                    '  #goodreads-widget {\n'
                    '    font-family: georgia, serif;\n'
                    '    padding: 18px 0;\n'
                    '    width:565px;\n'
                    '  }\n'
                    '  #goodreads-widget h1 {\n'
                    '    font-weight:normal;\n'
                    '    font-size: 16px;\n'
                    '    border-bottom: 1px solid #BBB596;\n'
                    '    margin-bottom: 0;\n'
                    '  }\n'
                    '  #goodreads-widget a {\n'
                    '    text-decoration: none;\n'
                    '    color:#660;\n'
                    '  }\n'
                    '  iframe{\n'
                    '    background-color: #fff;\n'
                    '  }\n'
                    '  #goodreads-widget a:hover { text-decoration: underline; '
                    '}\n'
                    '  #goodreads-widget a:active {\n'
                    '    color

                    '    border-bottom: 1px solid #BBB596;\n'
                    '    margin-bottom: 0;\n'
                    '  }\n'
                    '  #goodreads-widget a {\n'
                    '    text-decoration: none;\n'
                    '    color:#660;\n'
                    '  }\n'
                    '  iframe{\n'
                    '    background-color: #fff;\n'
                    '  }\n'
                    '  #goodreads-widget a:hover { text-decoration: underline; '
                    '}\n'
                    '  #goodreads-widget a:active {\n'
                    '    color:#660;\n'
                    '  }\n'
                    '  #gr_footer {\n'
                    '    width: 100%;\n'
                    '    border-top: 1px solid #BBB596;\n'
                    '    text-align: right;\n'
                    '  }\n'
                    '  #goodreads-widget .gr_branding{\n'
                    '    color: #382110;\n'
                    '    font-size: 1

                    '    text-decoration: none;\n'
                    '    font-family: "Helvetica Neue", Helvetica, Arial, '
                    'sans-serif;\n'
                    '  }\n'
                    '</style>\n'
                    '<div id="goodreads-widget">\n'
                    '  <div id="gr_header"><h1><a rel="nofollow" '
                    'href="https://www.goodreads.com/book/show/34433755-girls-of-paper-and-fire">Girls '
                    'of Paper and Fire Reviews</a></h1></div>\n'
                    '  <iframe id="the_iframe" '
                    'src="https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&amp;format=html&amp;isbn=0316561363&amp;links=660&amp;review_back=fff&amp;stars=000&amp;text=000" '
                    'width="565" height="400" frameborder="0"></iframe>\n'
                    '  <div id="gr_footer">\n'
                    '    <a class="gr_branding" target="_blank" rel="nofollow '
                    'noopener noreferrer"

 {'reviews_widget': '<style>\n'
                    '  #goodreads-widget {\n'
                    '    font-family: georgia, serif;\n'
                    '    padding: 18px 0;\n'
                    '    width:565px;\n'
                    '  }\n'
                    '  #goodreads-widget h1 {\n'
                    '    font-weight:normal;\n'
                    '    font-size: 16px;\n'
                    '    border-bottom: 1px solid #BBB596;\n'
                    '    margin-bottom: 0;\n'
                    '  }\n'
                    '  #goodreads-widget a {\n'
                    '    text-decoration: none;\n'
                    '    color:#660;\n'
                    '  }\n'
                    '  iframe{\n'
                    '    background-color: #fff;\n'
                    '  }\n'
                    '  #goodreads-widget a:hover { text-decoration: underline; '
                    '}\n'
                    '  #goodreads-widget a:active {\n'
                    '    color

                    '  #goodreads-widget a:active {\n'
                    '    color:#660;\n'
                    '  }\n'
                    '  #gr_footer {\n'
                    '    width: 100%;\n'
                    '    border-top: 1px solid #BBB596;\n'
                    '    text-align: right;\n'
                    '  }\n'
                    '  #goodreads-widget .gr_branding{\n'
                    '    color: #382110;\n'
                    '    font-size: 11px;\n'
                    '    text-decoration: none;\n'
                    '    font-family: "Helvetica Neue", Helvetica, Arial, '
                    'sans-serif;\n'
                    '  }\n'
                    '</style>\n'
                    '<div id="goodreads-widget">\n'
                    '  <div id="gr_header"><h1><a rel="nofollow" '
                    'href="https://www.goodreads.com/book/show/29010395-i-am-not-your-perfect-mexican-daughter">I '
                    'Am Not Your Perfect Mexican Daughter

...

                    '    color:#660;\n'
                    '  }\n'
                    '  #gr_footer {\n'
                    '    width: 100%;\n'
                    '    border-top: 1px solid #BBB596;\n'
                    '    text-align: right;\n'
                    '  }\n'
                    '  #goodreads-widget .gr_branding{\n'
                    '    color: #382110;\n'
                    '    font-size: 11px;\n'
                    '    text-decoration: none;\n'
                    '    font-family: "Helvetica Neue", Helvetica, Arial, '
                    'sans-serif;\n'
                    '  }\n'
                    '</style>\n'
                    '<div id="goodreads-widget">\n'
                    '  <div id="gr_header"><h1><a rel="nofollow" '
                    'href="https://www.goodreads.com/book/show/30613790-dan-and-phil-go-outside">Dan '
                    'and Phil Go Outside Reviews</a></h1></div>\n'
                    '  <iframe id="the_iframe" '
         

                    '  <div id="gr_footer">\n'
                    '    <a class="gr_branding" target="_blank" rel="nofollow '
                    'noopener noreferrer" '
                    'href="https://www.goodreads.com/book/show/29236298-gemina?utm_medium=api&amp;utm_source=reviews_widget">Reviews '
                    'from Goodreads.com</a>\n'
                    '  </div>\n'
                    '</div>\n'},
 {'reviews_widget': '<style>\n'
                    '  #goodreads-widget {\n'
                    '    font-family: georgia, serif;\n'
                    '    padding: 18px 0;\n'
                    '    width:565px;\n'
                    '  }\n'
                    '  #goodreads-widget h1 {\n'
                    '    font-weight:normal;\n'
                    '    font-size: 16px;\n'
                    '    border-bottom: 1px solid #BBB596;\n'
                    '    margin-bottom: 0;\n'
                    '  }\n'
                    '  #goodreads-widget a {\n'
      

                    '    color:#660;\n'
                    '  }\n'
                    '  iframe{\n'
                    '    background-color: #fff;\n'
                    '  }\n'
                    '  #goodreads-widget a:hover { text-decoration: underline; '
                    '}\n'
                    '  #goodreads-widget a:active {\n'
                    '    color:#660;\n'
                    '  }\n'
                    '  #gr_footer {\n'
                    '    width: 100%;\n'
                    '    border-top: 1px solid #BBB596;\n'
                    '    text-align: right;\n'
                    '  }\n'
                    '  #goodreads-widget .gr_branding{\n'
                    '    color: #382110;\n'
                    '    font-size: 11px;\n'
                    '    text-decoration: none;\n'
                    '    font-family: "Helvetica Neue", Helvetica, Arial, '
                    'sans-serif;\n'
                    '  }\n'
                    '</style>\n'
   

                    '</style>\n'
                    '<div id="goodreads-widget">\n'
                    '  <div id="gr_header"><h1><a rel="nofollow" '
                    'href="https://www.goodreads.com/book/show/25890355-paper-and-fire">Paper '
                    'and Fire Reviews</a></h1></div>\n'
                    '  <iframe id="the_iframe" '
                    'src="https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&amp;format=html&amp;isbn=0451472403&amp;links=660&amp;review_back=fff&amp;stars=000&amp;text=000" '
                    'width="565" height="400" frameborder="0"></iframe>\n'
                    '  <div id="gr_footer">\n'
                    '    <a class="gr_branding" target="_blank" rel="nofollow '
                    'noopener noreferrer" '
                    'href="https://www.goodreads.com/book/show/25890355-paper-and-fire?utm_medium=api&amp;utm_source=reviews_widget">Reviews '
                    'from Goodreads.com</a>\n'
                

                    '  #goodreads-widget h1 {\n'
                    '    font-weight:normal;\n'
                    '    font-size: 16px;\n'
                    '    border-bottom: 1px solid #BBB596;\n'
                    '    margin-bottom: 0;\n'
                    '  }\n'
                    '  #goodreads-widget a {\n'
                    '    text-decoration: none;\n'
                    '    color:#660;\n'
                    '  }\n'
                    '  iframe{\n'
                    '    background-color: #fff;\n'
                    '  }\n'
                    '  #goodreads-widget a:hover { text-decoration: underline; '
                    '}\n'
                    '  #goodreads-widget a:active {\n'
                    '    color:#660;\n'
                    '  }\n'
                    '  #gr_footer {\n'
                    '    width: 100%;\n'
                    '    border-top: 1px solid #BBB596;\n'
                    '    text-align: right;\n'
                    '  }\